In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
from sklearn.impute import KNNImputer
import time
from sklearn import preprocessing
from sklearn.preprocessing import LabelEncoder
import matplotlib.pyplot as plt

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
df=pd.read_csv('/content/drive/MyDrive/ML project/weatherAUS.csv')

In [ ]:
df.head()

,Date,Location,MinTemp,MaxTemp,Rainfall,Evaporation,Sunshine,WindGustDir,WindGustSpeed,WindDir9am,...,Humidity9am,Humidity3pm,Pressure9am,Pressure3pm,Cloud9am,Cloud3pm,Temp9am,Temp3pm,RainToday,RainTomorrow
0,2008-12-01,Albury,13.4,22.9,0.6,NaN,NaN,W,44.0,W,...,71.0,22.0,1007.7,1007.1,8.0,NaN,16.9,21.8,No,No
1,2008-12-02,Albury,7.4,25.1,0.0,NaN,NaN,WNW,44.0,NNW,...,44.0,25.0,1010.6,1007.8,NaN,NaN,17.2,24.3,No,No
2,2008-12-03,Albury,12.9,25.7,0.0,NaN,NaN,WSW,46.0,W,...,38.0,30.0,1007.6,1008.7,NaN,2.0,21.0,23.2,No,No
3,2008-12-04,Albury,9.2,28.0,0.0,NaN,NaN,NE,24.0,SE,...,45.0,16.0,1017.6,1012.8,NaN,NaN,18.1,26.5,No,No
4,2008-12-05,Albury,17.5,32.3,1.0,NaN,NaN,W,41.0,ENE,...,82.0,33.0,1010.8,1006.0,7.0,8.0,17.8,29.7,No,No


## Removing null values

In [ ]:
null_counts = df.isnull().sum()
null_counts[null_counts > len(df)*0.3]

Evaporation    62790
Sunshine       69835
Cloud9am       55888
Cloud3pm       59358
dtype: int64

In [ ]:
df = df.drop(['Evaporation','Sunshine','Cloud9am','Cloud3pm'], axis = 1)

We can drop all rows where the target attribute is missing

In [ ]:
df = df[df['RainTomorrow'].notnull()]

We can drop column Date

In [ ]:
df = df.drop(['Date'], axis = 1)

In [ ]:
len(df.dropna())/len(df)

0.7941670827677875

Clearly we can't simply drop all rows with missing values since we lose 21% of the dataset

In [ ]:
df['RainToday'] = (df['RainToday'] == 'Yes').astype('uint32')
df['RainTomorrow'] = (df['RainTomorrow'] == 'Yes').astype('uint32')

We can use KNN Imputer for predicting values of continuous variables

In [ ]:
continuous_df = df[['MinTemp', 'MaxTemp', 'Rainfall', 'WindGustSpeed', 'WindSpeed9am','WindSpeed3pm', 'Humidity9am', 'Humidity3pm', 'Pressure9am', 'Pressure3pm', 'Temp9am', 'Temp3pm', 'RainToday', 'RainTomorrow']]

252s to run next cell (with GPU)

In [ ]:
t = time.time()
imputer = KNNImputer(n_neighbors=3, weights='uniform', metric='nan_euclidean')
imputer.fit(continuous_df)
continuous_df_trans = pd.DataFrame(imputer.transform(continuous_df), columns = continuous_df.columns)
print(time.time()-t, "s to run")

242.8812117576599 s to run


In [ ]:
for col in continuous_df_trans.columns:
  df[col] = np.array(continuous_df_trans[col])

In [ ]:
df

,Location,MinTemp,MaxTemp,Rainfall,WindGustDir,WindGustSpeed,WindDir9am,WindDir3pm,WindSpeed9am,WindSpeed3pm,Humidity9am,Humidity3pm,Pressure9am,Pressure3pm,Temp9am,Temp3pm,RainToday,RainTomorrow
0,Albury,13.4,22.9,0.6,W,44.0,W,WNW,20.0,24.0,71.0,22.0,1007.7,1007.1,16.9,21.8,0.0,0.0
1,Albury,7.4,25.1,0.0,WNW,44.0,NNW,WSW,4.0,22.0,44.0,25.0,1010.6,1007.8,17.2,24.3,0.0,0.0
2,Albury,12.9,25.7,0.0,WSW,46.0,W,WSW,19.0,26.0,38.0,30.0,1007.6,1008.7,21.0,23.2,0.0,0.0
3,Albury,9.2,28.0,0.0,NE,24.0,SE,E,11.0,9.0,45.0,16.0,1017.6,1012.8,18.1,26.5,0.0,0.0
4,Albury,17.5,32.3,1.0,W,41.0,ENE,NW,7.0,20.0,82.0,33.0,1010.8,1006.0,17.8,29.7,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
145454,Uluru,3.5,21.8,0.0,E,31.0,ESE,E,15.0,13.0,59.0,27.0,1024.7,1021.2,9.4,20.9,0.0,0.0
145455,Uluru,2.8,23.4,0.0,E,31.0,SE,ENE,13.0,11.0,51.0,24.0,1024.6,1020.3,10.1,22.4,0.0,0.0
145456,Uluru,3.6,25.3,0.0,NNW,22.0,SE,N,13.0,9.0,56.0,21.0,1023.5,1019.1,10.9,24.5,0.0,0.0
145457,Uluru,5.4,26.9,0.0,N,37.0,SE,WNW,9.0,9.0,53.0,24.0,1021.0,1016.8,12.5,26.1,0.0,0.0


In [ ]:
df.isnull().sum()

Location             0
MinTemp              0
MaxTemp              0
Rainfall             0
WindGustDir       9330
WindGustSpeed        0
WindDir9am       10013
WindDir3pm        3778
WindSpeed9am         0
WindSpeed3pm         0
Humidity9am          0
Humidity3pm          0
Pressure9am          0
Pressure3pm          0
Temp9am              0
Temp3pm              0
RainToday            0
RainTomorrow         0
dtype: int64

In [ ]:
# KEEP THIS COMMENTED IF KNN IMPUTER RAN IN PREVIOUS CELL
# for col in continuous_df.columns:
#   df[col] = df[col].fillna(df[col].median())

In [ ]:
df

,Location,MinTemp,MaxTemp,Rainfall,WindGustDir,WindGustSpeed,WindDir9am,WindDir3pm,WindSpeed9am,WindSpeed3pm,Humidity9am,Humidity3pm,Pressure9am,Pressure3pm,Temp9am,Temp3pm,RainToday,RainTomorrow
0,Albury,13.4,22.9,0.6,W,44.0,W,WNW,20.0,24.0,71.0,22.0,1007.7,1007.1,16.9,21.8,0.0,0.0
1,Albury,7.4,25.1,0.0,WNW,44.0,NNW,WSW,4.0,22.0,44.0,25.0,1010.6,1007.8,17.2,24.3,0.0,0.0
2,Albury,12.9,25.7,0.0,WSW,46.0,W,WSW,19.0,26.0,38.0,30.0,1007.6,1008.7,21.0,23.2,0.0,0.0
3,Albury,9.2,28.0,0.0,NE,24.0,SE,E,11.0,9.0,45.0,16.0,1017.6,1012.8,18.1,26.5,0.0,0.0
4,Albury,17.5,32.3,1.0,W,41.0,ENE,NW,7.0,20.0,82.0,33.0,1010.8,1006.0,17.8,29.7,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
145454,Uluru,3.5,21.8,0.0,E,31.0,ESE,E,15.0,13.0,59.0,27.0,1024.7,1021.2,9.4,20.9,0.0,0.0
145455,Uluru,2.8,23.4,0.0,E,31.0,SE,ENE,13.0,11.0,51.0,24.0,1024.6,1020.3,10.1,22.4,0.0,0.0
145456,Uluru,3.6,25.3,0.0,NNW,22.0,SE,N,13.0,9.0,56.0,21.0,1023.5,1019.1,10.9,24.5,0.0,0.0
145457,Uluru,5.4,26.9,0.0,N,37.0,SE,WNW,9.0,9.0,53.0,24.0,1021.0,1016.8,12.5,26.1,0.0,0.0


In [ ]:
df['WindGustDir'].mode()

0    W
dtype: object

In [ ]:
df['WindDir9am'].mode()

0    N
dtype: object

In [ ]:
df['WindDir3pm'].mode()

0    SE
dtype: object

147 s to run next cell(with GPU)

In [ ]:
df['WindGustDir'] = df['WindGustDir'].fillna("W")
df['WindDir9am'] = df['WindDir9am'].fillna("N")
df['WindDir3pm'] = df['WindDir3pm'].fillna("SE")

In [ ]:
df = df.drop("Location", axis = 1)

In [ ]:
df.to_csv("/content/drive/MyDrive/ML project/weatherAUS-Non_null.csv", index = False)

In [ ]:
df.isnull().sum()

MinTemp          0
MaxTemp          0
Rainfall         0
WindGustDir      0
WindGustSpeed    0
WindDir9am       0
WindDir3pm       0
WindSpeed9am     0
WindSpeed3pm     0
Humidity9am      0
Humidity3pm      0
Pressure9am      0
Pressure3pm      0
Temp9am          0
Temp3pm          0
RainToday        0
RainTomorrow     0
dtype: int64

## Removing duplicates and OHE

In [ ]:
new_df = pd.read_csv("/content/drive/MyDrive/ML project/weatherAUS-Non_null.csv")

In [ ]:
new_df.drop_duplicates(keep='first', inplace=True)

In [ ]:
new_df.isnull().sum()

MinTemp          0
MaxTemp          0
Rainfall         0
WindGustDir      0
WindGustSpeed    0
WindDir9am       0
WindDir3pm       0
WindSpeed9am     0
WindSpeed3pm     0
Humidity9am      0
Humidity3pm      0
Pressure9am      0
Pressure3pm      0
Temp9am          0
Temp3pm          0
RainToday        0
RainTomorrow     0
dtype: int64

In [ ]:
new_df['RainToday'] = new_df['RainToday'].astype(int)
new_df['RainTomorrow'] = new_df['RainTomorrow'].astype(int)

The categorical columns are: \
  - WindGustDir 
  - WindDir9am
  - WindDir3pm

In [ ]:
new_df['WindGustDir'].value_counts()

W      19058
SE      9309
E       9071
N       9033
SSE     8993
S       8949
WSW     8901
SW      8797
SSW     8610
WNW     8066
NW      8003
ENE     7992
ESE     7305
NE      7060
NNW     6561
NNE     6433
Name: WindGustDir, dtype: int64

In [ ]:
new_df['WindDir9am'].value_counts()

N      21354
SE      9162
E       9024
SSE     8966
NW      8552
S       8493
W       8260
SW      8237
NNE     7948
NNW     7840
ENE     7735
ESE     7558
NE      7527
SSW     7448
WNW     7194
WSW     6843
Name: WindDir9am, dtype: int64

In [ ]:
new_df['WindDir3pm'].value_counts()

SE     14389
W       9911
S       9598
WSW     9329
SW      9182
SSE     9142
N       8667
WNW     8656
NW      8468
ESE     8382
E       8342
NE      8164
SSW     8010
NNW     7733
ENE     7724
NNE     6444
Name: WindDir3pm, dtype: int64

Since each of the columns that are categorical have a uniform distribution, aggregating less frequent vlaues into one value "other" won't be correct

One-hot encoding will result in 16 dimensions added for each of the 3 columns. 

Let's aggregate the 16 different values into 4 and then apply one-hot encoding

In [ ]:
cols_categorical = ['WindDir3pm', 'WindDir9am', 'WindGustDir']

for col in cols_categorical:
  new_vals = []
  for val in new_df[col]:
    if val in ['NNW','N', 'NNE','NE']:
      new_vals.append('N')
    elif val in ['ENE','E', 'ESE','SE']:
      new_vals.append('E')
    elif val in ['SSE','S', 'SSW','SW']:
      new_vals.append('S')
    elif val in ['WSW','W', 'WNW','NW']:
      new_vals.append('W')
    else:
      print("HIII:    ", val)
  
  new_df[col] = np.array(new_vals)

In [ ]:
new_df['WindGustDir'].value_counts()

W    44028
S    35349
E    33677
N    29087
Name: WindGustDir, dtype: int64

In [ ]:
new_df['WindDir9am'].value_counts()

N    44669
E    33479
S    33144
W    30849
Name: WindDir9am, dtype: int64

In [ ]:
new_df['WindDir3pm'].value_counts()

E    38837
W    36364
S    35932
N    31008
Name: WindDir3pm, dtype: int64

In [ ]:
one_hot_encoded_data = pd.get_dummies(new_df, columns = ['WindGustDir','WindDir9am', 'WindDir3pm'])

In [ ]:
one_hot_encoded_data

,MinTemp,MaxTemp,Rainfall,WindGustSpeed,WindSpeed9am,WindSpeed3pm,Humidity9am,Humidity3pm,Pressure9am,Pressure3pm,...,WindGustDir_S,WindGustDir_W,WindDir9am_E,WindDir9am_N,WindDir9am_S,WindDir9am_W,WindDir3pm_E,WindDir3pm_N,WindDir3pm_S,WindDir3pm_W
0,13.4,22.9,0.6,44.0,20.0,24.0,71.0,22.0,1007.7,1007.1,...,0,1,0,0,0,1,0,0,0,1
1,7.4,25.1,0.0,44.0,4.0,22.0,44.0,25.0,1010.6,1007.8,...,0,1,0,1,0,0,0,0,0,1
2,12.9,25.7,0.0,46.0,19.0,26.0,38.0,30.0,1007.6,1008.7,...,0,1,0,0,0,1,0,0,0,1
3,9.2,28.0,0.0,24.0,11.0,9.0,45.0,16.0,1017.6,1012.8,...,0,0,1,0,0,0,1,0,0,0
4,17.5,32.3,1.0,41.0,7.0,20.0,82.0,33.0,1010.8,1006.0,...,0,1,1,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
142188,3.5,21.8,0.0,31.0,15.0,13.0,59.0,27.0,1024.7,1021.2,...,0,0,1,0,0,0,1,0,0,0
142189,2.8,23.4,0.0,31.0,13.0,11.0,51.0,24.0,1024.6,1020.3,...,0,0,1,0,0,0,1,0,0,0
142190,3.6,25.3,0.0,22.0,13.0,9.0,56.0,21.0,1023.5,1019.1,...,0,0,1,0,0,0,0,1,0,0
142191,5.4,26.9,0.0,37.0,9.0,9.0,53.0,24.0,1021.0,1016.8,...,0,0,1,0,0,0,0,0,0,1


In [ ]:
one_hot_encoded_data = one_hot_encoded_data.reset_index()
one_hot_encoded_data = one_hot_encoded_data.drop('index', axis = 1)

In [ ]:
one_hot_encoded_data.to_csv("/content/drive/MyDrive/ML project/weatherAUS-clean.csv", index = False)

In [ ]:
one_hot_encoded_data

,MinTemp,MaxTemp,Rainfall,WindGustSpeed,WindSpeed9am,WindSpeed3pm,Humidity9am,Humidity3pm,Pressure9am,Pressure3pm,...,WindGustDir_S,WindGustDir_W,WindDir9am_E,WindDir9am_N,WindDir9am_S,WindDir9am_W,WindDir3pm_E,WindDir3pm_N,WindDir3pm_S,WindDir3pm_W
0,13.4,22.9,0.6,44.0,20.0,24.0,71.0,22.0,1007.7,1007.1,...,0,1,0,0,0,1,0,0,0,1
1,7.4,25.1,0.0,44.0,4.0,22.0,44.0,25.0,1010.6,1007.8,...,0,1,0,1,0,0,0,0,0,1
2,12.9,25.7,0.0,46.0,19.0,26.0,38.0,30.0,1007.6,1008.7,...,0,1,0,0,0,1,0,0,0,1
3,9.2,28.0,0.0,24.0,11.0,9.0,45.0,16.0,1017.6,1012.8,...,0,0,1,0,0,0,1,0,0,0
4,17.5,32.3,1.0,41.0,7.0,20.0,82.0,33.0,1010.8,1006.0,...,0,1,1,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
142136,3.5,21.8,0.0,31.0,15.0,13.0,59.0,27.0,1024.7,1021.2,...,0,0,1,0,0,0,1,0,0,0
142137,2.8,23.4,0.0,31.0,13.0,11.0,51.0,24.0,1024.6,1020.3,...,0,0,1,0,0,0,1,0,0,0
142138,3.6,25.3,0.0,22.0,13.0,9.0,56.0,21.0,1023.5,1019.1,...,0,0,1,0,0,0,0,1,0,0
142139,5.4,26.9,0.0,37.0,9.0,9.0,53.0,24.0,1021.0,1016.8,...,0,0,1,0,0,0,0,0,0,1
